In [ ]:
# default_exp measures

# Behavioural Measures

> Compute behavioural measures from raw transaction data

## Basics

A behavioural measure is a number derived from a player's transaction data. This data needs to have specific columns depending on the measure you want to calculate. For example, measures in the bet domain require the **bet_size** column. Before defining any measures, the first step is to create a checker method which makes sure that the data used has the right columns.

In [ ]:
#export
def check_measure_data(player_bets, required_columns):
    "Compares the columns found in a dataframe of player bets to a supplied list of column names. If any of the required_column names are not found, an exception is raised reporting the error."
    for column in required_columns:
        if column not in player_bets.columns:
            exception_string = (
                "Column '" + column + "' missing from provided dataframe."
            )
            raise Exception(exception_string)

## Root Domain

The root domain consists of behavioural measures that can be computed using only the knowledge of when the bets took place. This means using only values from the ***bet_time*** column in the player bet dataframe.

In [ ]:
#export
def duration(player_bets):
    "The number of days between the first bet and the last."
    check_measure_data(player_bets, ["bet_time"])
    player_bets.sort_values("bet_time", inplace=True)
    player_bets.reset_index(drop=True, inplace=True)

    first_day = player_bets.iloc[0]["bet_time"]
    last_day = player_bets.iloc[len(player_bets) - 1]["bet_time"]
    # add one to make it interpret as 'days where betting has occurred'
    age_in_days = (last_day.date() - first_day.date()).days + 1
    return age_in_days

## Bet Domain

The bet domain contains all behavioural measures which can be computed using additional values from the ***bet_size*** column. As such, they describe *spending* behaviours as opposed to *engagement* behaviours as above.

## Loss Domain

The loss domain includes all behavioural measures which require the additional information of the size of the payout received as a result of each bet. This data should be held in the ***payout_size*** column if available.

## Other Domain

All measures not in the root, bet, or loss domains, require some additional information such as the house edge for each bet, the game being played, etc.

## References

{% bibliography --cited %}